In [1]:
class SshConfig 
{
    public $username;
    public $password;
    public $hostname;
    public $port;
    
    public function __construct(
        string $username, string $password, string $hostname, int $port
    )
    {
        $this->username = $username;
        $this->password = $password;
        $this->hostname = $hostname;
        $this->port = $port;
    }
}

class MysqlConfig {
    public $username;
    public $password;
    public $hostname;
    public $port;
    
    public function __construct(
        string $username, string $password, string $hostname, int $port
    )
    {
        $this->username = $username;
        $this->password = $password;
        $this->hostname = $hostname;
        $this->port = $port;
    }
}

In [2]:
class SshTunneledDb2Connection 
{
    public $ssh_config;
    public $mysql_config;
    
    public function __construct(SshConfig $ssh_config, MysqlConfig $mysql_config)
    {
        $this->ssh_config = $ssh_config;
        $this->mysql_config = $mysql_config;
    }
    
    public function query(string $query)
    {        
        $connection = ssh2_connect($this->ssh_config->hostname, $this->ssh_config->port);
        if($connection === FALSE){
            throw new RuntimeException("Failed to connect with SSH server.");
        }

        if(!ssh2_auth_password($connection, $this->ssh_config->username, $this->ssh_config->password)){
              throw new RuntimeException("Failed to authenticate to SSH server");
        }
        
        $ssh_query ='ssh -L 3307:' . $this->ssh_config->hostname.':' .$this->mysql_config->port.'; 
        echo "' . str_replace( '"', '\'', stripslashes( $query ) ) . '" | mysql -u '.$this->mysql_config->username.' -h '.$this->mysql_config->hostname.' --password='.$this->mysql_config->password;
    
        $response = ssh2_exec($connection, $ssh_query);
        
        $error = ssh2_fetch_stream($response, SSH2_STREAM_STDERR);
        $error_response = stream_get_contents($error);

        stream_set_blocking($response, true);
        stream_set_blocking($error, true);
        
        // The 1st row of the result is the column names
        // The data is from 2nd row onwards
        // Each row is separated by \n
        // Data for columns within each row is separated by \t
        while($line = fgets($response)) {
            flush();
            echo $line;
        }
    }
}

class SshTunneledConnectionFactory
{
    static public function create_from_environment()
    {
        $ssh_config = new SshConfig(
            getenv('tunnel_ssh_username'),
            getenv('tunnel_ssh_password'),
            getenv('tunnel_ssh_host'),
            getenv('tunnel_ssh_port')
        );
        
        $mysql_config = new MysqlConfig(
            getenv('db_user'),
            getenv('db_password'),
            getenv('db_host'),
            getenv('tunnel_remote_bind_port')
        );
        
        return new SshTunneledDb2Connection($ssh_config, $mysql_config);
    }
}

$connection = SshTunneledConnectionFactory::create_from_environment();
print_r($connection->query("SELECT id, username FROM nanohub.jos_users LIMIT 10"));

id	username



211547	-100005



211574	-100024



211576	-100026



211583	-100032



...

211679	-100098



79194	-1001



211684	-100103



211688	-100107



true

In [3]:
// $ssh['username'] = getenv('tunnel_ssh_username');
// $ssh['password'] = getenv('tunnel_ssh_password');
// $ssh['hostname'] = getenv('tunnel_ssh_host');
// $ssh['port'] = getenv('tunnel_ssh_port');

// $mysql['username'] = getenv('db_user');
// $mysql['password'] = getenv('db_password');
// $mysql['hostname'] = getenv('db_host');
// $mysql['port'] = getenv('tunnel_remote_bind_port');

In [4]:
$query = 'SELECT * FROM nanohub.jos_faq';

$ssh_query ='ssh -L 3307:' . getenv('tunnel_ssh_host').':' . getenv('tunnel_remote_bind_port').'; 
echo "' . str_replace( '"', '\'', stripslashes( $query ) ) . '" | mysql -u '.getenv('db_user').' -h '.getenv('db_host').' --password='.getenv('db_password');

$response = ssh2_exec($connection, $ssh_query);
$error_response = ssh2_fetch_stream($response, SSH2_STREAM_STDERR);
print_r(stream_get_contents($error_response));

stream_set_blocking($response, true);
stream_set_blocking($error_response, true);

$results = stream_get_contents($response);
if (is_null($results ))
    echo "Empty response/result";

else 
    print_r($results);

ssh2_disconnect ( $connection );

PHP Warning:  ssh2_exec() expects parameter 1 to be resource, object given in /home/saxenap/.jupyter-php/pkgs/vendor/litipk/jupyter-php/src/Actions/ExecuteAction.php(115) : eval()'d code on line 4